In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [17]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]
)

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [18]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()

        # FC
        self.fc1 = nn.Linear(28 * 28, 128)  # 28x28 (784)
        self.relu1 = nn.ReLU()

        # FC
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()

        # FC
        self.fc3 = nn.Linear(64, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # x.shape -> (28x28 = 784)
        
        x = self.fc1(x)
        x = self.relu1(x)
        
        x = self.fc2(x)
        x = self.relu2(x)
        
        x = self.fc3(x)
        y = self.softmax(x)
        
        return y

In [19]:
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


model.train()
for epoch in range(5):
    correct = 0
    total = 0
    for inputs, labels in trainloader:
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
            
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch + 1}/{5}], "
          f"Loss: {loss.item()}, "
            f"Accuracy: {correct / total}")


Epoch [1/5], Loss: 1.555688738822937, Accuracy: 0.8337666666666667
Epoch [2/5], Loss: 1.4623059034347534, Accuracy: 0.91425
Epoch [3/5], Loss: 1.5509955883026123, Accuracy: 0.9296166666666666
Epoch [4/5], Loss: 1.516679286956787, Accuracy: 0.9389333333333333
Epoch [5/5], Loss: 1.5699670314788818, Accuracy: 0.9473


In [20]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
            
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {correct / total}")

Test Accuracy: 0.9517
